# Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.


### Load the data into Pandas dataframe

In [2]:
df = pd.read_csv('../input/movie-review/data.tsv',sep='\t',encoding='latin-1')
df.head()

Unnamed: 0                                             Phrase  new_sent
0      134302  , this is one adapted - from-television movie ...         1
1       49272  fails to give his audience a single character ...         0
2       51588  A dark comedy that goes for sick and demented ...         0
3        7399  there 's no indication that he 's been respons...         0
4      131842  is a such a brainless flibbertigibbet that it ...         0

Drop the columns that are not required for the neural network.

In [3]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 2 columns):
Phrase      12000 non-null object
new_sent    12000 non-null int64
dtypes: int64(1), object(1)
memory usage: 187.6+ KB


* Create input and output vectors.
* Process the labels.

In [5]:
X = df.Phrase
Y = df.new_sent
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

Split into training and test data.

In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

### Process the data
* Tokenize the data and convert the text to sequences.
* Add padding to ensure that all the sequences have the same shape.
* There are many ways of taking the *max_len* and here an arbitrary length of 150 is chosen.

In [7]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train) # I(4) am(5) aseem(78) -> 4 5 78
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

### LSTM (RNN)
Define the RNN structure.

In [13]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

Call the function and compile the model.

In [14]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

Fit on the training data.

In [15]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 7680 samples, validate on 1920 samples
Epoch 1/10
7680/7680 [==============================] - 19s 2ms/step - loss: 0.6064 - acc: 0.6665 - val_loss: 0.4844 - val_acc: 0.7729
Epoch 2/10
7680/7680 [==============================] - 18s 2ms/step - loss: 0.5091 - acc: 0.8014 - val_loss: 0.4577 - val_acc: 0.7917
Epoch 3/10
7680/7680 [==============================] - 18s 2ms/step - loss: 0.3860 - acc: 0.8307 - val_loss: 0.4686 - val_acc: 0.7932


The model performs well on the validation set and this configuration is chosen as the final model.

Process the test set data.

In [16]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

Evaluate the model on the test set.

In [17]:
accr = model.evaluate(test_sequences_matrix,Y_test)

2400/2400 [==============================] - 2s 900us/step


In [19]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],100*accr[1]))

Test set
  Loss: 0.446
  Accuracy: 79.333
